## Cumulative Utilization Rate of Machine

In [27]:
import sqlite3

def calculate_utilization(real_database_path, start_time_id, end_time_id, machine_id):
    # ----------- SUPPORTING FUNCTION -----------
    def update_utilization_end_id(real_database_path, start_time_id, end_time_id, machine_id):
        with sqlite3.connect(real_database_path) as db:
            #--- Read activity type (started and Finished) and event_id for a specific machien within a range
            read_machine_events = db.execute("SELECT activity_type, event_id FROM real_log WHERE machine_id= ? and event_id>=? AND event_id<=?",
                (machine_id, start_time_id, end_time_id)).fetchall()
            
            machine_acitivities= []
            machine_events_id = []
            for i in range(len(read_machine_events)):
                machine_acitivities.append(read_machine_events[i][0])
                machine_events_id.append(read_machine_events[i][1])

            #--- Flag to indicates if there is a finished before a start
            flag_last_is_started = False

            #--- Just the last value that matters, because for a specific machine either it's finish or started
            #------------ MACHINE LAST EVENT: STARTED -------------
            if machine_acitivities[-1] == 'Started':
                # If the last event is Started, it means that the previous one was Finished and tha
                # you should consider its end time id
                print(f"{machine_id} last trace is 'Started', updating the end time id from {end_time_id} to {machine_events_id[-2]}")
                end_time_id = machine_events_id[-2]
                flag_last_is_started = True

            if machine_acitivities[-1] == 'Finished':
                # Everything fine...
                pass

            return end_time_id
    
    #--- Update end time checking the last trace:
    end_time_id= update_utilization_end_id(real_database_path, start_time_id, end_time_id, machine_id)

    # connect to the database
    conn = sqlite3.connect(real_database_path)
    c = conn.cursor()

    # query the database for the machine's events within the given period
    c.execute("SELECT * FROM real_log WHERE machine_id=? AND event_id>=? AND event_id<=?",
              (machine_id, start_time_id, end_time_id))
    rows = c.fetchall()

    # calculate the total time the machine spent working within the given period
    total_working_time = 0
    last_start_time = None
    for row in rows:
        if row[3] == 'Started':
            last_start_time = row[7]
        elif row[3] == 'Finished' and last_start_time is not None:
            total_working_time += row[7] - last_start_time
            last_start_time = None
    
    # calculate the initial time
    c.execute("SELECT MIN(timestamp_real) FROM real_log WHERE event_id >= ? AND event_id <= ?", 
              (start_time_id, end_time_id))
    initial_time = c.fetchone()[0]

    # calculate the total time within the given period
    c.execute("SELECT MAX(timestamp_real) FROM real_log WHERE event_id>=? AND event_id<=?",
              (start_time_id, end_time_id))
    final_time = c.fetchone()[0] 

    # calculate the total time
    total_time = final_time - initial_time

    # calculate the utilization rate
    utilization_rate = (total_working_time / total_time) * 100

    print(f"--- Utilization Rate: {utilization_rate} %")
    print(f"--- Total Working Time: {total_working_time}")
    print(f"--- Total Time: {total_time}")

    # close the connection
    conn.close()

    return utilization_rate




In [29]:
exp_id = '3.30.19.26'
real_database_path = f'data_generation/{exp_id}/databases/real_database.db'
start_time_id = 1
end_time_id = 2
machine_id = 'Machine 2'

calculate_utilization(real_database_path, start_time_id, end_time_id, machine_id)

IndexError: list index out of range